In [ ]:
pip install transformers

In [ ]:
pip install seqeval

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 1. 데이터 로드

In [ ]:
import json
from google.colab import files

uploaded = files.upload()

Saving E2017.NEWS.001.json to E2017.NEWS.001.json
Saving E2017.NEWS.002.json to E2017.NEWS.002.json
Saving E2017.NEWS.003.json to E2017.NEWS.003.json
Saving E2017.NEWS.004.json to E2017.NEWS.004.json
Saving E2017.NEWS.005.json to E2017.NEWS.005.json
Saving E2017.NEWS.006.json to E2017.NEWS.006.json
Saving E2017.NEWS.007.json to E2017.NEWS.007.json
Saving E2017.NEWS.008.json to E2017.NEWS.008.json
Saving E2017.NEWS.009.json to E2017.NEWS.009.json
Saving E2017.NEWS.010.json to E2017.NEWS.010.json
Saving E2017.NEWS.011.json to E2017.NEWS.011.json
Saving E2017.NEWS.012.json to E2017.NEWS.012.json


In [ ]:
ner_label_file = files.upload()

Saving ner_label.txt to ner_label.txt


In [ ]:
# 문장과 레이블을 담는 리스트
sentences = []
labels = []

for i in range(1, 13, 1):
  # 파일 불러오기
  with open('E2017.NEWS.0{0:02d}.json'.format(i)) as data_file:
    local = json.load(data_file)

  # json 파일의 "sentence" 태그 안으로
  locals = local["sentence"]
  # locals 리스트 안에 각 딕션너리를 순서대로
  for locals_index in locals:
    # 각 딕션너리의 "NE" 태그 안으로
    locals_NEs = locals_index["NE"]

    # 한 문장 안에 있는 단어들과 그에 대한 레이블을 담는 리스트
    sentence = []
    label = []
    # locals_NEs 중 하나의 단어와 그에 대한 레이블
    for locals_NE in locals_NEs:
      text = locals_NE['text']
      type_ = locals_NE['type']

      # sentence 리스트 안에 text 추가
      sentence.append(text)
      # label 리스트 안에 type_ 추가
      label.append(type_)

    # sentence 리스트를 sentences 리스트에 추가
    sentences.append(sentence)
    # label 리스트를 labels 리스트에 추가
    labels.append(label)

for sentence in sentences:
  if not sentence:
    del sentence

# train 데이터와 test 데이터 분리
train_data_sentence, test_data_sentence, train_data_label, test_data_label = train_test_split(sentences, labels, test_size=0.2)

print(train_data_sentence)
print(train_data_label)
print(test_data_sentence)
print(test_data_label)

[['북한', '지난 4월 26일', '무인기', '성주'], ['2008년부터 2016년까지', '서울', '3.3㎡', '1748만 원', '1916만 원', '9.6%', 'GDP', '국내총생산', '1조 17억 달러', '1조 4110억 달러', '41%'], ['환자', '주민등록'], ['심폐소생술', '40분', '심장', '내과', '위진', '교수', '체외심폐순환기', 'ECMO'], ['순항미사일', '4월 15일', '평양', '김일성', '105주년', '열병식', '원통형', '4개', '미사일'], ['미국', '토요일', '오전 2시', '상담원'], ['연말', '18일부터'], ['윤인진', '고려대', '사회학과', '교수', '6개월∼1년', '사원', '고용제도'], ['북한', '중앙TV', '사드', '성주', '북쪽', 'X 대역탐지기', '서쪽'], ['경총', '새내기', '직장인', '1년', '49.1%'], ['한 달', '2억 9000만 원', '20.7%'], ['북한', '무인기', '북한'], ['ECMO', '환자'], ['20일 이후', '근로자'], ['미국', '50초', '상담원', '전화'], ['정', '한', '동창생', '대학', '학교'], ['북한', '무인기', '사드', '고고도 미사일방어체계', '경북', '성주', '북측'], [], ['상담원', '한국'], ['홍석현', '신임 특보', '서울', '경기고', '스탠포드대'], ['담임', '아빠', '딸'], ['인제', '무인기', '2∼3㎞', '성주', '북한', '지난달 8일', '조선중앙TV'], ['주말', '6시', '신한은행'], ['4월 초', '7억 8700만 원', '한 달', '7000만 원'], ['심폐소생술', '30분', '의사'], ['겨울철', '경찰'], ['경찰', '소래포구 어시장 화재', '목격자', '3명'], ['덕수상고', '행정고시', '입법고시'], ['아이돌', '연습생'

In [ ]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')] # label(정답) 파일을 열어서 labels을 가져옴.
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['PS_NAME', 'LC_OTHERS', 'LCP_COUNTRY', 'LCP_PROVINCE', 'LCP_COUNTY', 'LCP_CITY', 'LCP_CAPITALCITY', 'LCG_RIVER', 'LCG_OCEAN', 'LCG_BAY', 'LCG_MOUNTIAN', 'LCG_ISLAND', 'LCG_CONTINENT', 'LC_TOUR', 'LC_SPACE', 'OG_OTHERS', 'OGG_ECONOMY', 'OGG_EDUCATION', 'OGG_MILITARY', 'OGG_MEDIA', 'OGG_SPORTS', 'OGG_ART', 'OGG_MEDICINE', 'OGG_RELIGION', 'OGG_SCIENCE', 'OGG_LIBRARY', 'OGG_LAW', 'OGG_POLITICS', 'OGG_FOOD', 'OGG_HOTEL', 'AF_CULTURAL_ASSET', 'AF_BUILDING', 'AF_MUSICAL_INSTRUMENT', 'AF_ROAD', 'AF_WEAPON', 'AF_TRANSPORT', 'AF_WORKS', 'AFW_DOCUMENT', 'AFW_PERFORMANCE', 'AFW_VIDEO', 'AFW_ART_CRAFT', 'AFW_MUSIC', 'AF_WARES', 'DT_OTHERS', 'DT_DURATION', 'DT_DAY', 'DT_MONTH', 'DT_YEAR', 'DT_SEASON', 'DT_GEOAGE', 'DT_DYNASTY', 'TI_OTHERS', 'TI_DURATION', 'TI_HOUR', 'TI_MINUTE', 'TI_SECOND', 'CV_NAME', 'CV_TRIBE', 'CV_SPORTS', 'CE_SPORTS_INST', 'CV_POLICY', 'CV_TAX', 'CV_FUNDS', 'CV_LANGUAGE', 'CV_BUILDING_TYPE', 'CV_FOOD', 'CV_DRINK', 'CV_CLOTHING', 'CV_POSITION', 'CV_RELATION', 'CV_OC

In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)} # 태그를 인덱스로 표현하는 변수
index_to_tag = {index: tag for index, tag in enumerate(labels)} # 인덱스를 태그로 표현하는 변수

In [ ]:
tag_size = len(tag_to_index) # 태그의 개수
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 145


# 2. 전처리 예시

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base") # BertTokenizer 에서 이미 학습되어 있는 'klue/bert-base' 모델을 가져옴.

# 3. 전처리

In [ ]:
#? 샘플을 특징으로 변환
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token # cls_token : BERT가 분류 문제를 풀기 위해 추가된 특별 토큰. 모든 문장의 가장 첫 번째(문장의 시작) 토큰으로 삽입.
    sep_token = tokenizer.sep_token # sep_token : 각 문장을 구분하기 위해 사용되는 특별 토큰. 각 문장의 끝에 [SEP] 토큰이 사용됨.
    pad_token_id = tokenizer.pad_token_id # pad_token : 일정한 크기로 BERT에 입력하기 위해 사용되는 패딩 토큰.

    #? input_ids : input의 id를 달아주기 위한 변수.
    # attention_masks : 불필요하게 패딩 토큰에 대해서 어텐션을 하지 않도록 실제 단어와 패딩 토큰을 구분할 수 있도록 알려주는 시퀀스 입력. 실제 단어 및 토큰 : 1 / 패딩 토큰 : 0 으로 표현.
    #? token_type_ids : 토큰의 타입에 대한 id.
    # data_labels : 데이터에 대한 레이블 집합.
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    # tqdm() : 진행 상황을 Progress Bar 형태로 확인 가능.
      # total : 총 반복 횟수
    # zip() : 두 리스트를 튜플 리스트로 합침.
    '''
        ex) for number, upper, lower in zip("12345", "ABCDE", "abcde"):
                print(number, upper, lower)

            (1, A, a)
            (2, B, b)
            (3, C, c)
            .
            .
            .
    '''
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # tokens, labels_ids
        tokens = []
        labels_ids = []

        # example 과 label 에서 단어 하나와 그 단어에 해당하는 label_token 을 가져옴. 
        for one_word, label_token in zip(example, label):
            # one_word 에서 토큰화 작업을 하여 subword_token 들로 분리.
            print()
            print(one_word)
            subword_tokens = tokenizer.tokenize(one_word)
            print(subword_tokens)

            # list.append(['Ping', 'Pong']) : 리스트 끝에 iterable 자체 그대로 넣음.
              # ex) list: ['Tick', 'Tock', 'Song', ['Ping', 'Pong']]
            # list.extend(['Ping', 'Pong']) : 리스트 끝에 iterable의 모든 항목을 넣음.
              # ex) list: ['Tick', 'Tock', 'Song', 'Ping', 'Pong']
            tokens.extend(subword_tokens)

            # labels_ids.extend() : 리스트 끝에 iterable의 모든 항목을 넣음.
            labels_ids.extend([tag_to_index[label_token]] + [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # 스페셜 토큰 : [CLS], [SEP]와 같은 토큰들을 의미.
        # 적어도 한 문장에는 [CLS] 1개와 [SEP] 1개가 존재.
        # '[CLS]', '감사합니다', '[SEP]'
        special_tokens_count = 2

        # 토큰의 개수가 최대 seq의 개수를 넘겼을 때,
        if len(tokens) > max_seq_len - special_tokens_count:
            # tokens 를 최대 seq의 개수(special_tokens의 개수를 포함하여)만큼 인덱스 슬라이싱.
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            # labels_ids 를 최대 seq의 개수(special_tokens의 개수를 포함하여)만큼 인덱스 슬라이싱. 
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        # 한 문장이 다 들어갔으므로, tokens 에 [sep_token]([SEP]) 을 추가.
        tokens += [sep_token]
        # sep_token 에 해당하는 id 추가.
        labels_ids += [pad_token_id_for_label]

        # 모든 문장의 앞에 존재하여 하므로, tokens 앞에 [cls_token]([CLS]) 을 추가.
        tokens = [cls_token] + tokens
        # cls_token 에 해당하는 id 추가.
        labels_ids = [pad_token_id_for_label] + labels_ids

        # tokens 를 ids 로 변환한 것은 input_id 에 넣음.
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        # 어텐션 마스크란?
        ''' 
            BERT가 어텐션 연산을 할 때, 불필요하게 패딩 토큰에 대해서 어텐션을 하지 않도록
            실제 단어와 패딩 토큰을 구분할 수 있도록 알려주는 입력.
            필기 17-02-10 참고.
        '''
        # input_id 길이만큼 attention_mask 에 1을 추가.
        attention_mask = [1] * len(input_id)
        # max_seq_len 보다 input_id 의 개수가 더 적은 경우, 패딩해야 하므로,
        padding_count = max_seq_len - len(input_id)
        # 패딩하기 위해서 padding id 를 padding_count 만큼 input_id 에 채워넣음.
        input_id = input_id + ([pad_token_id] * padding_count)
        # 어텐션 마스크에는 padding_count 만큼 0을 추가.
        attention_mask = attention_mask + ([0] * padding_count)
        #? token_type_id 에 max_seq_len 만큼 pad_token_id_for_segment 를 넣음.
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        #? label 은 현재까지의 labels_ids 에 padding_count 만큼 pad_token_id_for_label 을 넣음.
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        # assert 는 뒤의 조건이 True 가 아니면 AssertError 를 발생.
        # input_id, attention_mask, token_type_id, label 의 크기가 max_seq_len 과 같으면, 에러를 발생.
        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        # input_ids 리스트에 input_id 리스트를 추가.
        input_ids.append(input_id)
        # attention_masks 리스트에 attention_mask 리스트를 추가.
        attention_masks.append(attention_mask)
        # token_type_ids 리스트에 token_type_id 리스트를 추가.
        token_type_ids.append(token_type_id)
        # data_labels 리스트에 label 리스트를 추가.
        data_labels.append(label)
    # 반복문 종료

    # input_ids, attention_masks, token_type_ids 리스트를 np.array() 로 새롭게 만듦.
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    # np.asarray() 는 데이터 형태가 다를 때만 복사. 데이터 형태가 같을 때 복사하지 않음.
    #? data_labels 리스트를 np.asarray() 로 새롭게 만듦. 데이터 타입을 np.int32 로 새롭게 만듦.
    data_labels = np.asarray(data_labels, dtype=np.int32)

    # 전처리를 수행한 결과를 리턴.
    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
# 위에서 만든 전처리 함수를 통해 훈련용 데이터를 가공.
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

 12%|█▎        | 32/256 [00:00<00:00, 475.66it/s]


북한
['북한']

지난 4월 26일
['지난', '4', '##월', '26', '##일']

무인기
['무인기']

성주
['성주']

2008년부터 2016년까지
['2008', '##년', '##부터', '2016', '##년', '##까', '##지']

서울
['서울']

3.3㎡
['3', '.', '3', '##㎡']

1748만 원
['174', '##8', '##만', '원']

1916만 원
['1916', '##만', '원']

9.6%
['9', '.', '6', '%']

GDP
['GDP']

국내총생산
['국내', '##총', '##생산']

1조 17억 달러
['1', '##조', '17', '##억', '달러']

1조 4110억 달러
['1', '##조', '41', '##10', '##억', '달러']

41%
['41', '%']

환자
['환자']

주민등록
['주민', '##등', '##록']

심폐소생술
['심폐', '##소', '##생', '##술']

40분
['40', '##분']

심장
['심장']

내과
['내과']

위진
['위', '##진']

교수
['교수']

체외심폐순환기
['체', '##외', '##심', '##폐', '##순환', '##기']

ECMO
['EC', '##M', '##O']

순항미사일
['순항', '##미사', '##일']

4월 15일
['4', '##월', '15', '##일']

평양
['평양']

김일성
['김일성']

105주년
['105', '##주', '##년']

열병식
['열병', '##식']

원통형
['원통', '##형']

4개
['4', '##개']

미사일
['미사일']

미국
['미국']

토요일
['토요일']

오전 2시
['오전', '2', '##시']

상담원
['상담원']

연말
['연말']

18일부터
['18', '##일', '##부터']

윤인진
['윤', '##인', '##진']

고려대
['고려대']

사회학과
['사회학', '##과'

KeyError: ignored

In [ ]:
# 위에서 만든 전처리 함수를 통해 테스트용 데이터를 가공.
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

  0%|          | 0/65 [00:00<?, ?it/s]


4대강
['4', '##대', '##강']


KeyError: ignored

# 4. 모델링

In [ ]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

# 5. 손실 함수

In [ ]:
def compute_loss(labels, logits):

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  active_loss = tf.reshape(labels, (-1,)) != -100
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

  return loss_fn(labels, reduced_logits)

# 6. 학습

In [ ]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.102.43.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.102.43.122:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():
  model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=compute_loss)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])
        
        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):

      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))

In [ ]:
f1_score_report = F1score(X_test, y_test)

In [ ]:
model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks = [f1_score_report]
)

Epoch 1/3


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None, 128) dtype=int32>]


Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None, 128) dtype=int32>]


   6/2532 [..............................] - ETA: 3:16 - loss: 2.2431WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0067s vs `on_train_batch_end` time: 7.8684s). Check your callbacks.


2532/2532 [==============================] - ETA: 0s - loss: 0.2726

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>]


 - f1: 84.91
              precision    recall  f1-score   support

         AFW       0.63      0.53      0.58       394
         ANM       0.71      0.76      0.73       701
         CVL       0.80      0.85      0.82      5758
         DAT       0.91      0.93      0.92      2521
         EVT       0.73      0.77      0.75      1094
         FLD       0.76      0.46      0.57       228
         LOC       0.82      0.86      0.84      2126
         MAT       0.22      0.17      0.19        12
         NUM       0.90      0.93      0.91      5590
         ORG       0.85      0.87      0.86      4086
         PER       0.89      0.89      0.89      4426
         PLT       0.42      0.15      0.22        34
         TIM       0.81      0.91      0.86       314
         TRM       0.74      0.73      0.74      1964

   micro avg       0.84      0.86      0.85     29248
   macro avg       0.73      0.70      0.71     29248
weighted avg       0.84      0.86      0.85     29248

2532/2532 [=

# 5. 예측

In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask


        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 635.44it/s]


In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

In [ ]:
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

In [ ]:
test_samples = [sent1, sent2]

In [ ]:
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 2/2 [00:00<00:00, 701.68it/s]


In [ ]:
result_list

[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀이다', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아', 'O'),
  ('있는데', 'O'),
  ('타인은', 'O'),
  ('살고', 'O'),
  ('있어요', 'O')]]